In [3]:
import torch

In [5]:
x = torch.empty(5, 3)
print(x)

tensor([[ 0.0000e+00,  0.0000e+00,  7.9033e-43],
        [ 0.0000e+00,  7.0065e-45,  0.0000e+00],
        [-3.4021e+18,  9.4307e-43,  0.0000e+00],
        [ 0.0000e+00,  7.0065e-45,  0.0000e+00],
        [-3.4024e+18,  9.4307e-43,  0.0000e+00]])


In [6]:
x = torch.rand(5, 3)
print(x)

tensor([[ 0.4261,  0.1225,  0.2303],
        [ 0.9008,  0.8498,  0.4787],
        [ 0.5600,  0.3563,  0.9716],
        [ 0.9238,  0.8096,  0.3307],
        [ 0.2103,  0.6650,  0.3210]])


In [7]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]])


In [8]:
x = torch.tensor([5.5, 3])
print(x)

tensor([ 5.5000,  3.0000])


In [9]:
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)                                      # result has the same size

tensor([[ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.]], dtype=torch.float64)
tensor([[-1.1617, -0.4659, -0.9928],
        [ 1.1055,  1.2501, -1.5914],
        [ 0.1636, -0.0107,  1.6876],
        [ 0.6641,  0.2743, -0.5208],
        [-0.8474,  1.5807, -1.0328]])


In [10]:
y = torch.rand(5, 3)
print(x + y)

tensor([[-0.7227,  0.2620, -0.9720],
        [ 1.2393,  1.8942, -1.1488],
        [ 0.5108,  0.6987,  2.1611],
        [ 1.4494,  1.0314, -0.1350],
        [-0.5344,  2.3308, -0.7796]])


In [13]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

tensor([[-0.1617,  0.5341,  0.0072],
        [ 2.1055,  2.2501, -0.5914],
        [ 1.1636,  0.9893,  2.6876],
        [ 1.6641,  1.2743,  0.4792],
        [ 0.1526,  2.5807, -0.0328]], device='cuda:0')
tensor([[-0.1617,  0.5341,  0.0072],
        [ 2.1055,  2.2501, -0.5914],
        [ 1.1636,  0.9893,  2.6876],
        [ 1.6641,  1.2743,  0.4792],
        [ 0.1526,  2.5807, -0.0328]], dtype=torch.float64)


In [24]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
y = x + 2
print(y)
print(y.grad_fn)

tensor([[ 1.,  1.],
        [ 1.,  1.]])
tensor([[ 3.,  3.],
        [ 3.,  3.]])


In [25]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[ 27.,  27.],
        [ 27.,  27.]]) tensor(27.)


In [26]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [28]:
out.backward()
print(x.grad)

tensor([[ 4.5000,  4.5000],
        [ 4.5000,  4.5000]])


In [42]:
## CNN Time

In [43]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [44]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [45]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0685, -0.1024,  0.0537, -0.0197, -0.0116, -0.0420, -0.1285,
          0.0708,  0.1085,  0.0814]])


In [46]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [47]:
output = net(input)
target = torch.arange(1, 11)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)
print(loss.grad_fn)

tensor(38.3148)


In [48]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [49]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update